In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab

import os
import json
import zipfile
import cv2
import imageio


pylab.rcParams['figure.figsize'] = (10.0, 8.0)

#img_io = imageio.imread('../images/testdev2017/000000466319.jpg')
#plt.imshow(img_io)
#plt.show()

In [13]:
annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print('Running demo for *%s* results.'%(annType))
print(prefix) # checking

Running demo for *bbox* results.
instances


In [40]:
#initialize COCO ground truth api
print(os.getcwd())
dataDir='../'
dataType='val2017'
annFile = '%sannotations/%s_%s.json'%(dataDir,prefix,dataType)

print(annFile) # checking

with open(annFile, 'r') as f:
    gt = json.load(f)

print(gt['images'][999])  # checking

cocoGt=COCO(annFile)
#print(cocoGt[0]) # checking

# load COCO categories
cats = cocoGt.loadCats(cocoGt.getCatIds())

# get all images containing given categories, select one at random
#catIds = cocoGt.getCatIds(catNms=['person'])
#imgIds = cocoGt.getImgIds(catIds=catIds)
#imgIds = cocoGt.getImgIds(imgIds=[1])
#img = cocoGt.loadImgs(imgIds[np.random.randint(0, len(imgIds))])[0]
#img


C:\Users\gordo\Desktop\CenterNet\data\coco\PythonAPI
../annotations/instances_val2017.json
{'license': 3, 'file_name': '000000508602.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000508602.jpg', 'height': 429, 'width': 500, 'date_captured': '2013-11-15 06:51:06', 'flickr_url': 'http://farm1.staticflickr.com/31/89958837_3946184e5f_z.jpg', 'id': 508602}
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!


In [27]:
#initialize COCO detections api
resFile='../results/results.json'
with open(resFile, 'r') as f:
    results = json.load(f)

print(results[0]) # testing
classes=[cat['name'] for cat in cats]    
    
det_count = 0
det_list = []
for item in results:
    if item["image_id"] == 16 and item["score"] >= 0.5:
        det_count = det_count + 1
        det_list.append((classes[item["category_id"]-1], item["score"], tuple(item["bbox"])))
        #print("Detection {}. = ".format(det_count) + str((classes[item["category_id"]-1], item["score"], tuple(item["bbox"]))))
print(det_list)

#resFile = resFile%(dataDir, prefix, dataType, annType)
cocoDt=cocoGt.loadRes(resFile)

{'image_id': 466319, 'category_id': 1, 'bbox': [199.02, 90.37, 135.19, 336.08], 'score': 0.94}
[('person', 0.95, (147.09, 126.52, 326.81, 503.52)), ('person', 0.63, (0.52, 198.63, 38.77, 315.76)), ('skateboard', 0.69, (237.94, 93.35, 24.51, 30.33)), ('tennis racket', 0.82, (34.69, 255.2, 212.23, 75.14))]
Loading and preparing results...
DONE (t=11.42s)
creating index...
index created!


In [19]:
imgIds = sorted(cocoGt.getImgIds())
imgIds = imgIds[0:100]
imgId = imgIds[np.random.randint(100)]

#imgIds = cocoGt.getImgIds(imgIds = [466319])
#img = cocoGt.loadImgs(imgIds)
#file_name = img[0]['file_name']

#print(os.getcwd())
#I = io.imread('../images/testdev2017/000000466319.jpg')

#plt.axis('off')
#plt.imshow(I)
#plt.show()


# load and display instance annotations
#annIds = cocoGt.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
#anns = cocoGt.loadAnns(annIds)
#coco.showAnns(anns)

#plt.imshow(I)
#plt.axis('off')
#annIds = cocoGt.getAnnIds(imgIds=img[0]['id'], catIds=catIds, iscrowd=None)
#print(annIds)
#anns = cocoGt.loadAnns(annIds)
#cocoGt.showAnns(anns)

In [20]:
# running evaluation
cocoEval = COCOeval(cocoGt, cocoDt, annType)
cocoEval.params.imgIds = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.63s).
Accumulating evaluation results...
DONE (t=0.17s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | m

None
